<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prosta-sieć-kodująco-rozkodowująca-zdjęcia." data-toc-modified-id="Prosta-sieć-kodująco-rozkodowująca-zdjęcia.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prosta sieć kodująco-rozkodowująca zdjęcia.</a></span><ul class="toc-item"><li><span><a href="#Dane---mnist" data-toc-modified-id="Dane---mnist-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dane - mnist</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Trening" data-toc-modified-id="Trening-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Trening</a></span></li><li><span><a href="#Rysujemy-efekty-kodowania" data-toc-modified-id="Rysujemy-efekty-kodowania-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Rysujemy efekty kodowania</a></span></li><li><span><a href="#Rysowanie-przestrzeni-kodowania" data-toc-modified-id="Rysowanie-przestrzeni-kodowania-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Rysowanie przestrzeni kodowania</a></span></li></ul></li><li><span><a href="#Rozszerzenie-kodowania-o-warstwy-konwolucyjne" data-toc-modified-id="Rozszerzenie-kodowania-o-warstwy-konwolucyjne-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Rozszerzenie kodowania o warstwy konwolucyjne</a></span><ul class="toc-item"><li><span><a href="#Model" data-toc-modified-id="Model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Trening" data-toc-modified-id="Trening-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Trening</a></span></li><li><span><a href="#Sprawdzenie-jakości" data-toc-modified-id="Sprawdzenie-jakości-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Sprawdzenie jakości</a></span></li><li><span><a href="#rysowanie-przestrzeni-kodowania" data-toc-modified-id="rysowanie-przestrzeni-kodowania-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>rysowanie przestrzeni kodowania</a></span></li></ul></li><li><span><a href="#Odszumianie-zdjęć" data-toc-modified-id="Odszumianie-zdjęć-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Odszumianie zdjęć</a></span><ul class="toc-item"><li><span><a href="#Zaszumianie-zdjęć---zadanie" data-toc-modified-id="Zaszumianie-zdjęć---zadanie-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Zaszumianie zdjęć - zadanie</a></span></li><li><span><a href="#Model-odszumiający" data-toc-modified-id="Model-odszumiający-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Model odszumiający</a></span></li><li><span><a href="#Trening" data-toc-modified-id="Trening-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Trening</a></span></li><li><span><a href="#Sprawdzanie-jakości-odszumiania" data-toc-modified-id="Sprawdzanie-jakości-odszumiania-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Sprawdzanie jakości odszumiania</a></span></li></ul></li><li><span><a href="#Tensorboard" data-toc-modified-id="Tensorboard-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tensorboard</a></span><ul class="toc-item"><li><span><a href="#Model-klasyfikatora---zadanie-ewentualnie" data-toc-modified-id="Model-klasyfikatora---zadanie-ewentualnie-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Model klasyfikatora - zadanie ewentualnie</a></span></li><li><span><a href="#Summary-i-Callback'i" data-toc-modified-id="Summary-i-Callback'i-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Summary i Callback'i</a></span></li><li><span><a href="#Trening" data-toc-modified-id="Trening-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Trening</a></span></li><li><span><a href="#Uruchomienie-tensorboard'a" data-toc-modified-id="Uruchomienie-tensorboard'a-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Uruchomienie tensorboard'a</a></span></li></ul></li></ul></div>

# __Unsupervised Deep Learning__
Podczas zajęć poznamy zastosowanie uczenia nienadzorowanego w deep learningu w celu zakodowania zdjęć a następnie wykorzystania tego samego mechanizmu do odszumiana zdjęć.

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow.keras import layers
from mpl_toolkits.mplot3d import Axes3D
import datetime
from sklearn.metrics import confusion_matrix
import itertools
import io
from sklearn.metrics import f1_score

# jeśli nie korzystasz z poniższej paczki, to wykomentuj te dwie linie poniżej
from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=False)

## Prosta sieć kodująco-rozkodowująca zdjęcia.
- skorzystamy z naszego ukochanego mnista ;)
- stworzymy sieć składającą się z dwóch części
    - enkodera, zejście z ilością cech w dół, aby jak najgęściej 'upakować' nasze dane
    - dekodera, zwiększanie ilości cech w celu uzyskania zdjęcia wejściowego w jak najmniej zmienionej postaci

### Dane - mnist

In [ ]:
# wczytujemy dane mnist'a z keras'a
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
# normalizujemy wartości pikseli z 0-255 do 0-1
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# podgląd kilku - znamy już to dobrze ;)
plt.matshow(x_train[0], cmap=cm.gray)
plt.show()
print(f"label: {y_train[0]}")

### Model
- po wejściu do modelu spłaszczamy zdjęcia to rozmiaru wektora 784
- enkoder będzie trzema warstwami fully connected z zmniejszającą się ilością neuronów
- dekoder będzie trzema warstwami fully connected z rosnącą ilością neuronów, aż do 784
- wychodząc zmieniamy kształ na zdjęcia 28,28

In [ ]:
# pusty kontener
model = tf.keras.models.Sequential()
# wejście i zmiana kształtu
model.add(layers.InputLayer(input_shape=(28,28)))
model.add(layers.Flatten())

############# enkoder #############
# kompresja zdjęcia do 256 wartości
model.add(layers.Dense(units=256, activation='relu'))
# kompresja do 128 wartości
model.add(layers.Dense(units=128, activation='relu'))
# kompresja do 64 wartości, to jest nasze zakodowane zdjęcie
model.add(layers.Dense(units=3, activation='relu', name='codedData'))

############# dekoder #############
# dekompresja do 128 wartości
model.add(layers.Dense(units=128, activation='relu'))
# dekompresja do 256 wartości
model.add(layers.Dense(units=256, activation='relu'))
# dekompresja do 784 wartości, wracamy do właściwej ilości
model.add(layers.Dense(units=784, activation='relu'))

# wyjściowow zmieniamy kształ na bardziej "zdjęciowy"
model.add(layers.Reshape((28,28)))

# wydruk podsumowania modelu
model.summary()

In [ ]:
# tworzymy optymalizator
#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)

In [ ]:
# kompilujemy model
model.compile(optimizer=opt, loss='mse')

### Trening

In [ ]:
# wykonujemy trening zapisując historię
history = model.fit(x_train, x_train, batch_size=200, epochs=10)

In [ ]:
# rysujemy wykres błędu
plt.plot(history.history["loss"])
plt.title("loss")
plt.show()

### Rysujemy efekty kodowania

In [ ]:
# funkcja do wyświetlania rekonstrukcji
def showReconstruction(data, labels, name, model):
    samplesNum = data.shape[0]
    xOut = model.predict(data)
    f,a=plt.subplots(2,samplesNum,figsize=(20,4))
    for i in range(samplesNum):
        a[0][i].matshow(data[i], cmap=cm.gray)
        a[0][i].axis('off')
        a[1][i].matshow(xOut[i], cmap=cm.gray)
        a[1][i].axis('off')
        a[1][i].set_title(labels[i])
    plt.suptitle(name +" data reconstruction")
    plt.show()

In [ ]:
# rekonstrukcja przykładów ze zbioru treningowego
recoNum = 10
xIn = x_train[-recoNum:]
yIn = y_train[-recoNum:]
showReconstruction(xIn, yIn, "Train", model)

In [ ]:
# rekonstrukcja przykładów ze zbioru testowego
recoNum = 20
xIn = x_test[-recoNum:]
yIn = y_test[-recoNum:]
showReconstruction(xIn, yIn, "Test", model)

### Rysowanie przestrzeni kodowania

In [ ]:
# tworzymy model częściowy, chcemy aby tylko zakodował zdjęcie
intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer("codedData").output)
intermediate_output = intermediate_layer_model.predict(x_train[:1000])

In [ ]:
# tworzymy rysunek 3D z zakodowanych danych
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(intermediate_output[:,0], intermediate_output[:,1], intermediate_output[:,2], c=y_train[:1000])
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [ ]:
# a teraz w 2D
plt.scatter(intermediate_output[:,0], intermediate_output[:,2], c=y_train[:1000])
plt.show()

## Rozszerzenie kodowania o warstwy konwolucyjne
- dodamy do modelu warstwy konwolucyjne
- przynajmniej jedna warstwa fully connected w środku
- ilość warstw i neuronów dowolna

### Model

In [ ]:
# pusty kontener
model = tf.keras.models.Sequential()
# wejście ze zmianą wymiarowości
model.add(layers.Reshape((28,28,1), input_shape=(28,28)))

############# enkoder #############
# ekstrakcja cech nr 1
model.add(layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))
# ekstrakcja cech nr 2
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))

# spłaszczenie
model.add(layers.Flatten())
model.add(layers.Dense(units=784, activation='relu'))
model.add(layers.Dense(units=392, activation='relu'))
model.add(layers.Dense(units=196, activation='relu'))
model.add(layers.Dense(units=98, activation='relu'))
model.add(layers.Dense(units=49, activation='relu'))
# kompresja do 3 wartości, to jest nasze zakodowane zdjęcie
model.add(layers.Dense(units=3, activation='relu', name='codedData'))

############# dekoder #############
model.add(layers.Dense(units=49, activation='relu'))
model.add(layers.Dense(units=98, activation='relu'))
model.add(layers.Dense(units=196, activation='relu'))
model.add(layers.Dense(units=392, activation='relu'))
model.add(layers.Dense(units=784, activation='relu'))
# dekompresja do 1568 wartości, jest to równowartość 7*7*32
model.add(layers.Dense(units=1568, activation='relu'))
# zmiana kształu pod konwolucje
model.add(layers.Reshape((7, 7, 32)))
# przejście przez konwolucje transponowane w celu zwiększenia rozmiaru zdjęcia
#7x7x32
model.add(layers.Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))
# przejście przez konwolucje transponowane w celu zwiększenia rozmiaru zdjęcia
#14x14x16
model.add(layers.Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))
#28x28x1
# wyjściowow zmieniamy kształ na bardziej "zdjęciowy"
model.add(layers.Reshape((28,28)))

# wydruk podsumowania modelu
model.summary()

In [ ]:
# tworzymy optymalizator
opt = tf.keras.optimizers.Adam(learning_rate=0.003)
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)

In [ ]:
# kompilujemy model
model.compile(optimizer=opt, loss='mse')

### Trening

In [ ]:
# wykonujemy trening zapisując historię
history = model.fit(x_train, x_train, batch_size=200, epochs=20)

In [ ]:
# rysujemy wykres błędu
plt.plot(history.history["loss"])
plt.title("loss")
plt.show()

### Sprawdzenie jakości

In [ ]:
# rekonstrukcja przykładów ze zbioru treningowego
recoNum = 10
xIn = x_train[-recoNum:]
yIn = y_train[-recoNum:]
showReconstruction(xIn, yIn, "Train", model)

In [ ]:
# rekonstrukcja przykładów ze zbioru testowego
recoNum = 20
xIn = x_test[-recoNum:]
yIn = y_test[-recoNum:]
showReconstruction(xIn, yIn, "Test", model)

### rysowanie przestrzeni kodowania

In [ ]:
# tworzymy model częściowy, chcemy aby tylko zakodował zdjęcie
intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer("codedData").output)
intermediate_output = intermediate_layer_model.predict(x_train[:1000])

In [ ]:
# tworzymy rysunek 3D z zakodowanych danych
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(intermediate_output[:,0], intermediate_output[:,1], intermediate_output[:,2], c=y_train[:1000])
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [ ]:
# a teraz w 2D
plt.scatter(intermediate_output[:,1], intermediate_output[:,2], c=y_train[:1000])
plt.show()

## Odszumianie zdjęć
- sztucnie 'zniszczymy' zdjęcia mnista
- sieć kodująco-rozkodująca się nie zmienia
- nowe jesty porównanie w trakcie treningu zdjęć odszumionych przez sieć z oryginalnymi

### Zaszumianie zdjęć - zadanie
- skorzystamy z numpy
- szum będzie losowymi liczbami
- dodajemy losowy szum do zdjęć/pikseli
- rysujemy przykładowe wyniki zaszumiania

In [ ]:
# funkcja zaszumiająca
def makeSomeNoise(images, noiseStrengh = 0.1):
    # pobieramy kształcałego zbioru do zaszumienia
    imgsShape = images.shape
    #print("imgs shape", imgsShape)
    # liczymy ile jest wszystkihc pikseli w zbiorze, do każdego osono coś dodamy
    pixelsNum = imgsShape[0]*imgsShape[1]*imgsShape[2]
    # generujemy szum i zmieniamy na kształ zbioru danych
    noise = np.random.normal(0,noiseStrengh,pixelsNum).reshape(imgsShape[0],imgsShape[1],imgsShape[2])
    # upewniamy się, że ilości się zgadzają
    #print("noise shape", noise.shape)
    # ostatecznie dodajemy szum do zdjęć
    imgsNoised = np.add(images, noise)
    return imgsNoised

In [ ]:
# funkcja do wyświetlania serii zdjęć
def showImages(data, name):
    samplesNum = data.shape[0]
    f,a=plt.subplots(1,samplesNum,figsize=(20,4))
    for i in range(samplesNum):
        a[i].matshow(data[i], cmap=cm.gray)
        a[i].axis('off')
    plt.suptitle(name)
    plt.show()

In [ ]:
showImages(x_train[:10], "Original MNIST")

In [ ]:
showImages(makeSomeNoise(x_train[:10], 0.3), "Noised MNIST")

In [ ]:
# zaszumiamy cały dataset
x_train_noised = makeSomeNoise(x_train, 0.3)
x_test_noised = makeSomeNoise(x_test, 0.5)

### Model odszumiający
- architektura modelu będzie podobna

In [ ]:
# pusty kontener
model = tf.keras.models.Sequential()
# wejście ze zmianą wymiarowości
model.add(layers.Reshape((28,28,1), input_shape=(28,28)))

############# enkoder #############
# ekstrakcja cech nr 1
model.add(layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))
# ekstrakcja cech nr 2
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))

# spłaszczenie
model.add(layers.Flatten())
model.add(layers.Dense(units=784, activation='relu'))
# kompresja do 3 wartości, to jest nasze zakodowane zdjęcie
model.add(layers.Dense(units=392, activation='relu'))

############# dekoder #############
model.add(layers.Dense(units=784, activation='relu'))
# dekompresja do 1568 wartości, jest to równowartość 7*7*32
model.add(layers.Dense(units=1568, activation='relu'))
# zmiana kształu pod konwolucje
model.add(layers.Reshape((7, 7, 32)))
# przejście przez konwolucje transponowane w celu zwiększenia rozmiaru zdjęcia
model.add(layers.Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))
# przejście przez konwolucje transponowane w celu zwiększenia rozmiaru zdjęcia
model.add(layers.Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu"))

# wyjściowow zmieniamy kształ na bardziej "zdjęciowy"
model.add(layers.Reshape((28,28)))

# wydruk podsumowania modelu
model.summary()

In [ ]:
# tworzymy optymalizator
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)

In [ ]:
# kompilujemy model
model.compile(optimizer=opt, loss='mse')

### Trening
- tym razem jako dane wejściowe podajemy zaszumione
- zaś jako wyjściowe podajemy oryginalne

In [ ]:
# wykonujemy trening zapisując historię
history = model.fit(x_train_noised, x_train, batch_size=200, epochs=40)

In [ ]:
# rysujemy wykres błędu
plt.plot(history.history["loss"])
plt.title("loss")
plt.show()

### Sprawdzanie jakości odszumiania

In [ ]:
howMany = 20
# oryginalne
showImages(x_train[:howMany], "Original MNIST")
# zaszumione
showImages(x_train_noised[:howMany], "Noised MNIST")
# oszumione
showImages(model.predict(x_train_noised[:howMany]), "Unnoised MNIST")

In [ ]:
howMany = 20
# oryginalne
showImages(x_test[:howMany], "Original MNIST")
# zaszumione
showImages(x_test_noised[:howMany], "Noised MNIST")
# oszumione
showImages(model.predict(x_test_noised[:howMany]), "Unnoised MNIST")

## Tensorboard
- poznamy narzędzie zintegrowane z tensorflow do wizualizacji
    - postępu uczenia w postaci wykresów błędu i jakości modelu
    - architektury modelu
    - wartości poszczególnych wag modelu
    - podglądu przykłądowych wejść oraz wyjść

### Model klasyfikatora - zadanie ewentualnie

In [ ]:
# kontener na model
model = tf.keras.models.Sequential()
# pierwsza warstwa zmienia kształ na odpowiadający pod konwolucje, czyli 4D, posiada również rozmiar wejścia
model.add(tf.keras.layers.Reshape(target_shape=(28,28,1), input_shape=(28,28)))
# pierwsza ukryta, konwolucyjna
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], padding="same"))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPool2D())
# druga ukryta, konwolucyjna
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=[3,3], padding="same"))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPool2D())
# trzecia ukryta, konwolucyjna
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], padding="same"))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPool2D())
# spłaszczamy pod warstwy fully connected(Dense)
model.add(tf.keras.layers.Flatten())
# czwarta ukryta fully connetced
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.3))
# wyjściowa, zadaniem jest klasyfikacja 10 klas
model.add(tf.keras.layers.Dense(10))
# aktywacja jako softmax, aby uzyskać rozkład prawdopodbieństwa
model.add(tf.keras.layers.Softmax())
# żadnego dropout'u, nie chcemy abym nam wyzerowało wartość

In [ ]:
# drukujemy opis modelu
model.summary()

In [ ]:
# wybieramy optimalizator, polecam pobawić się z różnymi typami oraz różnymi wartościami learning rate'u
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
# kompilujemy
model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=["accuracy"])

### Summary i Callback'i
- tworzymy zapisywanie do plików naszych wyników
- skorzystamy z wbudowanej funkcji w kerasa a także z ręcznie robionych callback'ów

In [ ]:
# nazwy klas
class_names = "0 1 2 3 4 5 6 7 8 9".split()
class_names

In [ ]:
# dodajemy zapisywanie logów do tensorboard'a
log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, )
# tworzymy obiekt zapisujący confusion matrix podczas treningu
imgs_dir = log_dir + "\\imgs"
file_writer_cm = tf.summary.create_file_writer(imgs_dir + '/cm')

In [ ]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

In [ ]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [ ]:
def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  y_pred = model.predict(x_test)
  y_pred = np.argmax(y_pred, axis=1)

  # Calculate the confusion matrix.
  cm = confusion_matrix(y_test, y_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

### Trening
- trenujemy i badamy różne ilości epok oraz rozmiary batchy
- test przy pomocy metody wbudowanej evaluate

In [ ]:
# trenujemy, warto sprawdzić z różną wielkością batcha i epok
history = model.fit(x_train, y_train, 
                    batch_size=1000, epochs=5, verbose=1, 
                    initial_epoch=0,
                    validation_data=(x_test, y_test), 
                    callbacks=[tensorboard_callback, cm_callback])

In [ ]:
# rysujemy wykres błędu
plt.plot(history.history["loss"])
plt.title("loss")
plt.show()

### Uruchomienie tensorboard'a

In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard

In [ ]:
# uruchomienie serwera, jeżeli z poziomu notebooka nie działa, to należy uruchomić z cmd, ale bez '%'
#%tensorboard --logdir "logs"